# Data

In [1]:
from custom_dataset import *

/home/yizhou/miniconda3/envs/cliport/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/resnet-18 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
batch_size = 8

In [3]:
train_dataset = HandleDataset4Cliport("/home/yizhou/Research/temp1/")
val_dataset = HandleDataset4Cliport("/home/yizhou/Research/temp1/", is_train = False)

100%|████████████████████████████████████████████████████████████████████| 174/174 [00:00<00:00, 2115.48it/s]


In [4]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)


In [5]:
# batch = next(iter(train_loader))
# x, y, l = batch 

# Model

In [6]:
from custom_cliport import *

In [7]:
model = CustomCliport()
model = model.cuda()

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
count_parameters(model)

10334257

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
from tqdm.auto import tqdm

# Train

In [12]:
total_step = 0
best_val_loss = 1e6

In [13]:
train_record, val_record = [], []

for epoch in range(20):
    train_loss_list = []
    for batch in tqdm(train_loader):
        total_step += 1

        x, y, l = batch

        batch_size = x.size(0)
        pred_y = model(x, l)
        
        # reshape
        y = y.view(batch_size, -1)
        pred_y = pred_y.view(batch_size, -1)
        
        optimizer.zero_grad()
        loss = (-y * F.log_softmax(pred_y, -1)).sum()
        loss.backward()
        optimizer.step()
        
        train_loss_list.append(loss.item())
    
    t_loss = np.mean(train_loss_list)
    print(epoch, "train loss: ", t_loss)
    train_record.append(t_loss)
    
    
    model.eval()
    val_loss_list = []
    for batch in tqdm(val_loader):
        total_step += 1

        x, y, l = batch

        batch_size = x.size(0)
        pred_y = model(x, l)
        
        # reshape
        y = y.view(batch_size, -1)
        pred_y = pred_y.view(batch_size, -1)
        
        loss = (-y * F.log_softmax(pred_y, -1)).sum()
        val_loss_list.append(loss.item())
    
    val_loss = np.mean(val_loss_list)
    print(epoch, "val loss: ", val_loss)
    val_record.append(val_loss)
    
    # save model
    if val_loss < best_val_loss:
        model_path = '/home/yizhou/Research/temp0/custom_cliport824.pth'
        torch.save(model.state_dict(), model_path)
        best_val_loss = val_loss 
    
    

100%|██████████████████████████████████████████████████████████████████████| 176/176 [00:06<00:00, 25.47it/s]


0 train loss:  12963.469114823774


100%|████████████████████████████████████████████████████████████████████████| 56/56 [00:01<00:00, 42.16it/s]


0 val loss:  11142.04137311663


100%|██████████████████████████████████████████████████████████████████████| 176/176 [00:06<00:00, 29.04it/s]


...

18 val loss:  10469.658987862724


100%|██████████████████████████████████████████████████████████████████████| 176/176 [00:06<00:00, 26.70it/s]


19 train loss:  8644.405297019264


100%|████████████████████████████████████████████████████████████████████████| 56/56 [00:01<00:00, 36.00it/s]

19 val loss:  10691.581394740513


In [14]:
train_record, val_record

([12963.469114823774,
  11320.019437616522,
  10696.415399724787,
  10232.879169117321,
  9832.788049871271,
  9730.367370605469,
  9503.330710671165,
  9423.198910799894,
  9317.683711658825,
  9219.6225530451,
  9129.913155295631,
  9026.955039284447,
  9009.190084284002,
  8894.650762384588,
  8826.616760253906,
  8888.28570834073,
  8817.477562644264,
  8811.96255493164,
  8686.841043645685,
  8644.405297019264],
 [11142.04137311663,
  10019.375091552734,
  9718.92440359933,
  9831.502964564732,
  9990.632690429688,
  9659.25940813337,
  10288.480643136161,
  9928.493974958148,
  10312.933968680245,
  10238.57157244001,
  10222.120082310268,
  9897.078077043805,
  10096.021824428013,
  10154.937526157924,
  10341.211321149554,
  10133.384312220982,
  10080.740931919643,
  10386.890568324498,
  10469.658987862724,
  10691.581394740513])

In [15]:
pause

NameError: name 'pause' is not defined

# Test

In [ ]:
# model_path = '/home/yizhou/Research/temp0/custom_cliport823.pth'
model.load_state_dict(torch.load(model_path))

In [ ]:
model.clip_text_feature_path = model.clip_text_feature_path.replace("json","pickle")

In [ ]:
import pickle

In [ ]:
text2clip_feature = pickle.load(open(model.clip_text_feature_path,'rb'))

In [ ]:
text2clip_feature = {k: v.tolist() for k, v in text2clip_feature.items()}

In [ ]:
# import json
# # the json file where the output must be stored
# out_file = open("text2clip_feature.json", "w")
  
# json.dump(text2clip_feature, out_file)

In [ ]:
batch = next(iter(val_loader))

In [ ]:
x, y, l = batch

In [ ]:
x.shape, y.shape, l.shape

In [ ]:
pred_y = model(x, l)

In [ ]:
pred_y = pred_y.view(pred_y.size(0), -1)

In [ ]:
pred_y = F.softmax(pred_y, dim = -1)

In [ ]:
pred_y = pred_y.view(-1, 256, 256)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(np.array(pred_y[11].cpu().data) * 255, cmap = "gray")

In [ ]:
plt.imshow(np.array(y[11].cpu().data) * 255, cmap = "gray")

In [ ]:
torch.argmax(y[3].cpu().data).item() // 256, torch.argmax(y[3].cpu().data).item() % 256

In [ ]:
torch.argmax(pred_y[3].cpu().data).item() // 256, torch.argmax(pred_y[3].cpu().data).item() % 256

In [ ]:
pred_max_index = torch.argmax(pred_y[11].cpu().data).item() 

h, w =  pred_max_index// 256, pred_max_index % 256
print(h,w)

# get direction
top_bound = max(h - 5, 0)
bottom_bound = min(h + 5, 255)

left_bound = max(w - 5, 0)
right_bound = min(w + 5, 255)

# mean over vertical direction
v_mean = torch.mean(pred_y[0][top_bound:bottom_bound, w]).item()
h_mean = torch.mean(pred_y[0][left_bound:right_bound, h]).item()

handle_dir = "horizontal" if v_mean > h_mean else "vertical" # if vertical direction more concentrate, then direciton is horizontal

print(handle_dir)